In [4]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import plotly.express as px

In [3]:
#ticker = yf.Ticker('TSM')
#upgrade_downgrades = ticker.upgrades_downgrades

#pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
#upgrade_downgrades[(upgrade_downgrades.index.year.isin([2024])) & (upgrade_downgrades['Firm'].isin(firms))]
#ticker.earnings_dates

,Firm,ToGrade,FromGrade,Action
GradeDate,,,,
2024-07-18 10:26:58,Needham,Buy,Buy,reit
2024-07-15 09:29:03,Needham,Buy,Buy,main
2024-06-20 11:26:56,Bernstein,Outperform,Outperform,main
2024-06-05 15:17:33,Barclays,Overweight,Overweight,main
2024-05-22 11:01:59,Needham,Buy,Buy,reit
2024-04-19 13:13:53,Barclays,Overweight,Overweight,main
2024-04-18 10:44:51,Needham,Buy,Buy,reit
2024-04-10 11:02:17,Needham,Buy,Buy,main
2024-04-02 18:53:07,Barclays,Overweight,Overweight,main


In [81]:
firms = [
    "Morgan Stanley",
    "Citigroup",
    #"Deutsche Bank",
    "Barclays",
    "JP Morgan",
    "UBS",
    "Jefferies",
    "Raymond James",
    "Bank of America",
    "Wells Fargo",
    "KeyBanc",
    "BMO Capital",
    "Goldman Sachs",
    "Oppenheimer",
    "Nomura",
    "Baird",
    "SunTrust Robinson Humphrey",
    "Canaccord Genuity",
    "PiperJaffray",
    "B of A Securities",
    "Wedbush",
    "Cowen & Co.",
    "Macquarie",
    "Needham",
    'Bernstein',
    'Stifel',
    'JMP Securities'
]

# needham buy -> outperform
# strong buy -> buy

rating_map = {
    'Buy': 1,
    'Strong Buy': 1,
    'Conviction Buy': 1,
    'Top Pick': 1,
    'Overweight': 2,
    'Outperform': 2,
    'Outperformer': 2,
    'Market Outperform': 2,
    'Market Perform': 3,
    'Hold': 3,
    'Neutral': 3,
    'Equal-Weight': 3,
    'Equal-weight': 3,
    'Sector Weight': 3,
    'Sector Perform': 3,
    'Perform': 3,
    'Underweight': 4,
    'Underperform': 4,
    'Underperformer': 4,
    'Market Underperform': 4,
    'Sell': 5,
    'Cautious': 5,
    'Reduce': 5,
    'In-line': 3,
    'In-Line': 3,
    'Above Average': 2
}

In [89]:
def prep_df(ticker):
    start = dt.datetime(2000, 1, 1)
    end = dt.datetime.now()
    df = yf.download(ticker, start=start, end=end)
    stock = yf.Ticker(ticker)
    upgrade_downgrades = stock.upgrades_downgrades
    earnings_dates = stock.earnings_dates
    earnings_dates.index = earnings_dates.index.tz_localize(None)

    earnings_dates = earnings_dates[(earnings_dates.index >= start) & (earnings_dates.index <= end)]
    #display(earnings_dates)
    upgrade_downgrades = upgrade_downgrades[(upgrade_downgrades['Firm'].isin(firms)) & (upgrade_downgrades.index >= start)]
    upgrade_downgrades['ToGrade'] = upgrade_downgrades['ToGrade'].map(rating_map)

    upgrade_downgrades.index = upgrade_downgrades.index.strftime('%Y-%m-%d')
    dates = (earnings_dates.index + pd.offsets.BusinessDay()).strftime('%Y-%m-%d')

    matching_dates = upgrade_downgrades.index.intersection(dates)
    result = upgrade_downgrades[upgrade_downgrades.index.isin(matching_dates)]
    display(result[result.Firm == 'UBS'])

    for i in range(len(dates) - 1, -1, -1):
        quarter = dates[i]
        good = result.loc[quarter]['ToGrade'].isin(positive_ratings).all()
        if good:
            start_price = df.loc[quarter]['Adj Close']
            if i != 0:
                end_date = pd.to_datetime(dates[i-1]) - pd.offsets.BusinessDay(2)
                end_price = df.loc[end_date]['Adj Close']
                print(f"{ticker} {quarter} to {end_date.strftime('%Y-%m-%d')} Returns: {(end_price / start_price - 1).round(4)}")

In [90]:
prep_df('AMZN')

[*********************100%%**********************]  1 of 1 completed
C:\Users\Bogdan\AppData\Local\Temp\ipykernel_22264\1243171410.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  upgrade_downgrades['ToGrade'] = upgrade_downgrades['ToGrade'].map(rating_map)


,Firm,ToGrade,FromGrade,Action
GradeDate,,,,
2024-05-01,UBS,1,Buy,main
2024-02-02,UBS,1,Buy,main
2023-08-04,UBS,1,Buy,main
2023-04-28,UBS,1,Buy,main
2023-02-03,UBS,1,Buy,main
2022-07-29,UBS,1,,main
